# Greenhouse Model

## Import dependencies and data

In [1]:
# import dependencies

import time
import os
import pickle
import json
from collections import deque
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from IPython.display import clear_output

In [2]:
# import prop. funcs and models

from data_funcs import k_fold_data_validation
from train_funcs import train_network, test_autoreg
from networks import feedForwardNeuralNetwork, recurrentNeuralNetwork
from opts import AdaGrad, RMSProp, Adam

In [3]:
# import external models

from statsmodels.tsa.arima.model import ARIMA
from sklearn.ensemble import GradientBoostingRegressor
# from xgboost import XGBRegressor

## Create data for specific fold

In [4]:
# define relevant paths
home_path = os.path.dirname(os.getcwd())
data_path = home_path + '\\data\\'
results_path = home_path + '\\nn_models\\results\\'

# get merged data
data = pd.read_csv(
    data_path + 'data_processed.csv',
    header=[0, 1],
    index_col=[0, 1, 2, 3, 4]
)

In [60]:
# define data specs
k_frac = 0.05
m = 5
t_steps = 1
n_steps = 6

# Define cols to use
cols = [
    ('temperatures', 'TA01_GT10X_GM10X'),
    ('temperatures', 'TA01_GT401_GM401'),
    ('temperatures', 'DC_GT301_damped'),
    ('sun', 'gsi'),
    ('power', 'phase'),
#     ('time', 'minofday'),
#     ('temperatures', 'TA01_GT10X_GM10X_loss'),
#     ('humidity', 'TA01_GT10X_GM10X_abs'),
#     ('humidity', 'TA01_GT401_GM401_abs'),
#     ('humidity', 'outdoor_abs'),
    ('temperatures', 'TA01_GT401_GM401_scaled'),
    ('temperatures', 'DC_GT301_damped_scaled'),
#     ('state', 'TA01_output'),
#     ('state', 'TA02_output')
]

## K-fold CV - Neural Nets

In [35]:
version = 31

In [38]:
# training hyperparams
k1 = 6
k2 = 1
lambd = 0.01
sigma = 1.0
seed = 1

# more params
n_epochs = 10
n_batch = 128
lr = 0.001

In [39]:
for k_idx in range(20):
    train_tup, test_tup, val_tup, col_params = k_fold_data_validation(
        data=data.copy(), 
        k_idx=k_idx, 
        k_frac=k_frac, 
        m=m, 
        cols=cols, 
        t_steps=t_steps, 
        n_steps=n_steps,
        setpoint=True,
        shuffle=False
    )
    
    # extract tuples
    (sequences_train, targets_train, temps_train, temps_t_train, _) = train_tup
    (sequences_test, targets_test, temps_test, temps_t_test, sequences_masked, _) = test_tup
    (sequences_val, targets_val, temps_val, temps_t_val, _) = val_tup
    
#     ### ----- FEED-FORWARD NN -----
#     # define network
#     model_name = 'FFNN_v{}_k{}'.format(version, k_idx)
#     units=[32, 32, 32]
#     model = feedForwardNeuralNetwork(
#         k1=k1,
#         k2=k2,
#         m=units,
#         seed=seed
#     )
    
#     # define optimizer
#     adam = Adam(
#         beta1=0.9,
#         beta2=0.999,
#         eps=1e-8,
#         weights=model.weights,
#     )
    
#     # set optimizer
#     model.optimizer = adam
    
#     # train model
#     results = train_network(
#         model=model,
#         train_data=(sequences_train[:, -2, :], temps_t_train, targets_train),
#         val_data=(sequences_val[:, -2, :], temps_t_val, targets_val),
#         seed=seed,
#         n_epochs=n_epochs,
#         n_batch=n_batch,
#         lambd=lambd,
#         sigma=sigma,
#         lr=lr,
#         optimizer='adam'
#     )
    
#     # get autoregressive test predictions
#     test_preds, test_encodings = test_autoreg(
#         model,
#         sequences_masked[:, -2, :],
#         temps_t_test,
#         targets_test,
#         t_steps
#     )
    
#     results['test_preds'] = test_preds
#     results['test_encodings'] = test_encodings
    
#     # save results
#     save_path = results_path + 'ffnn\\' + model_name + '.pickle'
#     with open(save_path, 'wb') as fo:
#         pickle.dump(results, fo)
    
#     # clear output
#     clear_output()
    
    
    ### ----- RECURRENT NN -----
    # define network
    model_name = 'RNN_v{}_k{}'.format(version, k_idx)
    units = 32
    model = recurrentNeuralNetwork(
        k1=k1,
        k2=k2,
        m=units,
        seed=seed
    )
    
    # define optimizer
    adam = Adam(
        beta1=0.9,
        beta2=0.999,
        eps=1e-8,
        weights=model.weights,
    )
    
    # set optimizer
    model.optimizer = adam
    
    # train model
    results = train_network(
        model=model,
        train_data=(sequences_train, temps_t_train, targets_train),
        val_data=(sequences_val, temps_t_val, targets_val),
        seed=seed,
        n_epochs=n_epochs,
        n_batch=n_batch,
        lambd=lambd,
        sigma=sigma,
        lr=lr,
        optimizer='adam'
    )
 
    # get autoregressive test predictions
    test_preds, test_encodings = test_autoreg(
        model,
        sequences_masked,
        temps_t_test,
        targets_test,
        t_steps
    )
    
    results['test_preds'] = test_preds
    results['test_encodings'] = test_encodings

    # save results
    save_path = results_path + 'rnn\\' + model_name + '.pickle'
    with open(save_path, 'wb') as fo:
        pickle.dump(results, fo)

    # clear output
    clear_output()
    

In [106]:
# # units=[32, 32, 32]
# # model = feedForwardNeuralNetwork(
# #     k1=k1,
# #     k2=k2,
# #     m=units,
# #     seed=seed
# # )

# units = 32
# model = recurrentNeuralNetwork(
#     k1=k1,
#     k2=k2,
#     m=units,
#     seed=seed
# )

# weight_count = 0
# for weight in model.weights.values():
#     weight_count += weight.size

## K-fold CV - other models

In [10]:
# for k_idx in range(20):
#     train_tup, test_tup, val_tup, col_params = k_fold_data_validation(
#         data=data.copy(), 
#         k_idx=k_idx, 
#         k_frac=k_frac, 
#         m=m, 
#         cols=cols, 
#         t_steps=t_steps, 
#         n_steps=n_steps,
#         setpoint=True,
#         shuffle=False
#     )
    
#     # extract tuples
#     (sequences_train, targets_train, temps_train, temps_t_train, _) = train_tup
#     (sequences_test, targets_test, temps_test, temps_t_test, sequences_masked, _) = test_tup
#     (sequences_val, targets_val, temps_val, temps_t_val, _) = val_tup

#     ### TRAIN ARIMA
#     clear_output()
#     print('TRAINING ARIMA, k_idx: {}'.format(k_idx))
    
#     # get model name
#     model_name = 'arima_v{}_k{}'.format(version, k_idx)
    
#     # get endogenous and exogenous regressors
#     endog_train = temps_t_train.tolist()
#     exog_train = [np.array(seq) for seq in sequences_train[:, -2, :].tolist()]
# #     exog_train = [np.array(seq).mean(axis=0) for seq in sequences_train[:, -t_steps:, :].tolist()]

#     endog_test = temps_t_test.tolist()
#     exog_test = [np.array(seq) for seq in sequences_masked[:, -2, :].tolist()]
# #     exog_test = [np.array(seq).mean(axis=0) for seq in sequences_test[:, -t_steps:, :].tolist()]
    
#     # estimate model
#     start = time.time()
#     arima_temp = ARIMA(endog=endog_train, exog=exog_train, order=(n_steps, 1, 0)).fit()
#     train_time = time.time() - start
    
#     # get predictions (t steps)
#     print('PREDICTING w. ARIMA, k_idx: {}'.format(k_idx))
#     arima_preds = []
#     for exog in exog_test:
#         pred = arima_temp.forecast(steps=1, exog=exog)
#         arima_preds.append(pred)

#     save_path = results_path + 'arima\\' + model_name
#     #arima_temp.save(save_path + '_model')
    
#     results = {
#         'test_preds':arima_preds,
#         'train_time':train_time
#     }
#     with open(save_path + '.pickle', 'wb') as fo:
#         pickle.dump(results, fo)
        
#     ### TRAIN GBDT
#     clear_output()
#     print('TRAINING GBDT, k_idx: {}'.format(k_idx))
    
#     # get model name
#     model_name = 'gbdt_v{}_k{}'.format(version, k_idx)
#     gbdt = GradientBoostingRegressor(
#         loss='squared_error',
#         learning_rate=0.001,
#         n_estimators=2000,
#         max_depth=10,
#         max_leaf_nodes=None,
#         #subsample=0.7,
#         verbose=1,
# #         n_iter_no_change=10,
# #         tol=1e-4,
# #         validation_fraction=0.1
#     )
    
#     # train GBDT
#     start = time.time()
#     X_train = np.hstack((np.stack(exog_train), temps_t_train))
#     gbdt.fit(X_train, targets_train.flatten())
#     train_time = time.time() - start
    
#     # create queue for preds
#     pred_queue = deque(maxlen=t_steps)
#     for temp in temps_t_test[:t_steps]:
#         pred_queue.append(temp)
    
#     # iterate over test seqs and get preds
#     gbdt_preds = []
#     X_test = [np.array(seq) for seq in sequences_masked[:, -2, :].tolist()]
#     X_test = np.hstack((np.stack(X_test), temps_t_test))

#     for x in X_test:
#         temp = np.array([pred_queue.popleft()])
#         x[-1] = temp[0][0]
#         temp_pred = gbdt.predict(x[np.newaxis, :])
#         pred_queue.append(temp_pred)
#         gbdt_preds.append(temp_pred)
        
#     # save model
#     save_path = results_path + 'gbdt\\' + model_name
# #     with open(save_path + '_model.pickle'.format(gbdt_name), 'wb') as fo:
# #         pickle.dump(gbdt, fo)
        
#     # save model results
#     results = {
#         'test_preds':gbdt_preds,
#         'train_loss':gbdt.train_score_,
#         'train_time':train_time,
#         'model_params':gbdt.get_params(),
#     }
#     with open(save_path + '.pickle', 'wb') as fo:
#         pickle.dump(results, fo)

# K-fold CV - GRU and LSTM

In [53]:
import tensorflow as tf
from tensorflow.keras import layers, Model

In [54]:
def get_model(gru_or_lstm, lr):
    tf.keras.backend.clear_session()

    inputs = layers.Input(shape=(sequences_train.shape[1], sequences_train.shape[2]))
    
    if gru_or_lstm.lower() == 'gru':
        _, encoder = layers.GRU(
            units=32, 
            recurrent_dropout=0.0, 
            return_sequences=False, 
            return_state=True, 
            activity_regularizer=tf.keras.regularizers.L2(0.01)
        )(inputs)
    else:
        _, _, encoder = layers.LSTM(
            units=32, 
            recurrent_dropout=0.0, 
            return_sequences=False, 
            return_state=True, 
            activity_regularizer=tf.keras.regularizers.L2(0.01)
        )(inputs)
        
    temp_input = layers.Input(shape=(1,))
    temp = layers.GaussianNoise(stddev=1.0)(temp_input)
    temp = layers.BatchNormalization()(temp)

    output = layers.Concatenate()([encoder, temp])
    output = layers.Dense(units=1, activation=None, use_bias=False, activity_regularizer=tf.keras.regularizers.L2(0.01))(output)

    model = Model([inputs, temp_input], output)
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='mse')
    return model

In [62]:
for k_idx in range(20):
    train_tup, test_tup, val_tup, col_params = k_fold_data_validation(
        data=data.copy(), 
        k_idx=k_idx, 
        k_frac=k_frac, 
        m=m, 
        cols=cols, 
        t_steps=t_steps, 
        n_steps=n_steps,
        setpoint=True,
        shuffle=False
    )
    
    # extract tuples
    (sequences_train, targets_train, temps_train, temps_t_train, _) = train_tup
    (sequences_test, targets_test, temps_test, temps_t_test, sequences_masked, _) = test_tup
    (sequences_val, targets_val, temps_val, temps_t_val, _) = val_tup

    ### TRAIN GRU
    clear_output()
    print('TRAINING GRU, k_idx: {}'.format(k_idx))
    
    # get model name
    model_name = 'gru_v{}_k{}'.format(version, k_idx)
    
    # get model
    gru_model = get_model('gru', lr=0.001)
    
    # train model
    trainHist = gru_model.fit(
        [sequences_train, temps_train],
        targets_train,
        epochs=30,
        batch_size=128,
        validation_split=0.0,
    )
    
    # get autoregressive preds and encodings
    preds = []
#     encodings = []
    
    pred_queue = deque(maxlen=t_steps)
    for temp in temps_test[:t_steps]:
        pred_queue.append(temp[np.newaxis, :])
        preds.append(temp)

    for seq in sequences_masked:
        temp = pred_queue.popleft()
        temp_pred = gru_model.predict([seq[np.newaxis, :], temp], verbose=0)
        pred_queue.append(temp_pred)
        preds.append(temp_pred[0])
#         encodings.append(encoded[0])

    results = {}
    results['test_preds'] = preds
    results['test_encodings'] = encodings
    # save results
    save_path = results_path + 'gru\\' + model_name + '.pickle'
    with open(save_path, 'wb') as fo:
        pickle.dump(results, fo)

    # clear output
    clear_output()
        
    ### TRAIN LSTM
    clear_output()
    print('TRAINING LSTM, k_idx: {}'.format(k_idx))
    
    # get model name
    model_name = 'lstm_v{}_k{}'.format(version, k_idx)
    
    # get model
    lstm_model = get_model('lstm', lr=0.001)
    
    # train model
    trainHist = lstm_model.fit(
        [sequences_train, temps_train],
        targets_train,
        epochs=30,
        batch_size=128,
        validation_split=0.0,
    )
    
    # get autoregressive preds and encodings
    preds = []
#     encodings = []

    pred_queue = deque(maxlen=t_steps)
    for temp in temps_test[:t_steps]:
        pred_queue.append(temp[np.newaxis, :])
        preds.append(temp)

    for seq in sequences_masked:
        temp = pred_queue.popleft()
        temp_pred = lstm_model.predict([seq[np.newaxis, :], temp], verbose=0)
        pred_queue.append(temp_pred)
        preds.append(temp_pred[0])
#         encodings.append(encoded[0])
        
    results = {}
    results['test_preds'] = preds
    results['test_encodings'] = encodings
    # save results
    save_path = results_path + 'lstm\\' + model_name + '.pickle'
    with open(save_path, 'wb') as fo:
        pickle.dump(results, fo)

TRAINING LSTM, k_idx: 3
Epoch 1/30
382/382 [==============================] - 5s 11ms/step - loss: 0.2213
Epoch 2/30
382/382 [==============================] - 4s 10ms/step - loss: 0.1213
Epoch 3/30
382/382 [==============================] - 4s 11ms/step - loss: 0.1024
Epoch 4/30
382/382 [==============================] - 4s 10ms/step - loss: 0.0898
Epoch 5/30
382/382 [==============================] - 4s 10ms/step - loss: 0.0824
Epoch 6/30
382/382 [==============================] - 4s 11ms/step - loss: 0.0767
Epoch 7/30
382/382 [==============================] - 4s 11ms/step - loss: 0.0729
Epoch 8/30
382/382 [==============================] - 4s 10ms/step - loss: 0.0696
Epoch 9/30
382/382 [==============================] - 4s 11ms/step - loss: 0.0671
Epoch 10/30
382/382 [==============================] - 4s 10ms/step - loss: 0.0655
Epoch 11/30
382/382 [==============================] - 4s 10ms/step - loss: 0.0634
Epoch 12/30
382/382 [==============================] - 4s 10ms/step - lo

KeyboardInterrupt: 